In [11]:
%%time
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE,RandomOverSampler
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from itertools import combinations 
from mlxtend.classifier import StackingClassifier
from sklearn import model_selection

CPU times: user 42 µs, sys: 9 µs, total: 51 µs
Wall time: 52.9 µs


In [12]:
df=pd.read_csv('60s_window_wrist_chest.csv',index_col=0)
df['label'].value_counts()

0    312
3    208
1    175
2     90
Name: label, dtype: int64

In [13]:
features=df.columns.tolist()
features

removed = ['label']
for rem in removed:
    features.remove(rem)

features_with_sub=[]
features_with_sub[:]=features

In [14]:
sm = SMOTE(random_state=2)
X, y= sm.fit_sample(df[features_with_sub], df['label'])
df_new=pd.concat([pd.DataFrame(X,columns=features_with_sub),pd.DataFrame(y,columns=['label'])],axis=1)
df_new

for i in range (len(list(df_new['subject']))):
    df_new['subject'][i] = min([2,3,4,5,6,7,8,9,10,11,13,14,15,16,17], key=lambda x:abs(x-df_new['subject'][i]))
df_new['subject']=df_new['subject'].astype(int)

/home/sf/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [15]:
df_new['label'].value_counts()

3    312
2    312
1    312
0    312
Name: label, dtype: int64

In [16]:
df_new[df_new['subject']==2]['label'].value_counts()

0    20
3    16
1    14
2    10
Name: label, dtype: int64

In [18]:
subject_list=[2,3,4,5,6,7,8,9,10,11,13,14,15,16,17]

In [31]:
data_list=[]
for i in subject_list:
    subject_df=df_new[df_new['subject']==i]
    X, y= sm.fit_sample(subject_df[features_with_sub], subject_df['label'])
    df_new_sm=pd.concat([pd.DataFrame(X,columns=features_with_sub),pd.DataFrame(y,columns=['label'])],axis=1)
    df_new_sm['subject']=df_new_sm['subject'].astype(int)
    if(i==2):
        data_list=(df_new_sm)
    else:
        data_list=pd.concat([data_list,df_new_sm],axis=0)
data_list.subject.unique()

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14, 15, 16, 17])

In [33]:
data_list[data_list['subject']==4]['label'].value_counts()

3    25
2    25
1    25
0    25
Name: label, dtype: int64

In [34]:
data_list['label'].value_counts()

3    367
2    367
1    367
0    367
Name: label, dtype: int64

In [35]:
len(data_list.columns.tolist())

72

In [36]:
removed = ['subject']
for rem in removed:
    features.remove(rem)

feature=features
print(len(feature))
len(features_with_sub)

70


71

In [38]:
p_d=pd.read_csv('personal_detail.csv',index_col=0)

df_new_1=data_list.merge(p_d,on='subject')
df_new_1

,net_acc_mean,net_acc_std,net_acc_min,net_acc_max,ACC_x_mean,ACC_x_std,ACC_x_min,ACC_x_max,ACC_y_mean,ACC_y_std,...,label,age,height,weight,gender_ female,coffee_today_YES,sport_today_YES,smoker_YES,feel_ill_today_YES,bmi
0,0.025961,0.013811,0.000000,0.087383,0.023431,0.017769,-0.037843,0.087383,0.000016,1.222594e-05,...,0,27,175,80,0,0,0,0,0,1
1,0.027640,0.010597,0.002752,0.054356,0.027640,0.010597,0.002752,0.054356,0.000019,7.290999e-06,...,0,27,175,80,0,0,0,0,0,1
2,0.028389,0.006937,0.000000,0.066053,0.028378,0.006985,-0.002752,0.066053,0.000020,4.805734e-06,...,0,27,175,80,0,0,0,0,0,1
3,0.033268,0.007670,0.000000,0.074998,0.032960,0.008904,-0.030962,0.074998,0.000023,6.126303e-06,...,0,27,175,80,0,0,0,0,0,1
4,0.037021,0.001284,0.027522,0.043347,0.037021,0.001284,0.027522,0.043347,0.000025,8.837530e-07,...,0,27,175,80,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,0.040139,0.000384,0.037016,0.043486,-0.040139,0.000384,-0.043486,-0.037016,-0.000028,2.641029e-07,...,3,29,165,55,1,0,0,0,0,0
1464,0.037988,0.006475,0.004223,0.056832,-0.037972,0.006549,-0.056832,0.001831,-0.000026,4.505862e-06,...,3,29,165,55,1,0,0,0,0,0
1465,0.032089,0.000321,0.030766,0.032902,-0.032089,0.000321,-0.032902,-0.030766,-0.000022,2.207868e-07,...,3,29,165,55,1,0,0,0,0,0
1466,0.030973,0.005802,0.001052,0.056056,-0.030966,0.005840,-0.056056,0.004372,-0.000021,4.018035e-06,...,3,29,165,55,1,0,0,0,0,0


In [39]:
df_new_1.to_csv('60s_window_wrist_chest_with_smote_all_subject_balenced_class.csv')